# Multiclass Text classification [link](https://www.tensorflow.org/tutorials/keras/text_classification)

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Downlaoding the dataset
url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

os.chdir('3-text-classifcation-dataset')
dataset = tf.keras.utils.get_file("stack_overflow_16k", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

In [3]:
# spliting train and test
BATCH_SIZE = 32
SEED = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'train', 
    batch_size=BATCH_SIZE, 
    validation_split=0.2, 
    subset='training', 
    seed=SEED)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'train', 
    batch_size=BATCH_SIZE, 
    validation_split=0.2, 
    subset='validation', 
    seed=SEED)

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'test', 
    batch_size=BATCH_SIZE)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.


2023-02-19 01:04:50.627832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-19 01:04:50.636000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-19 01:04:50.636447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-19 01:04:50.637353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [4]:
# preparig traing data (converting it to one hot encoding)
MAX_FEATURE = 10000
MAX_LENTH = 250

def custom_standarization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    stripped_html = tf.strings.regex_replace(stripped_html, '\n', '')
    return tf.strings.regex_replace(stripped_html, # removing punctuation
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
# x = next(iter(raw_train_ds)) # tupple (text->(batch), class)
# print(custom_standarization(x[0][0]))


# vectorization layer (convertinig text to one-hot-encoding)
vectorize_layer = layers.TextVectorization(
    standardize=custom_standarization,
    max_tokens=MAX_FEATURE, # the lenth of the one-hot vector
    output_mode='int',
    output_sequence_length=MAX_LENTH #the max number of tokens in the input sentence
    
)


def vectorize_text(text, label):
    '''
    this function will be mapped (applyed to the dataset) as a prameter
    '''
    tf.expand_dims(text, -1)
    return vectorize_layer(text), label

# Initializing the vocabulary of the TextVectorizaion layer
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


# Applying the vectorization
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [5]:
x = next(iter(raw_train_ds)) # tupple (text->(batch), class)
print(x[1][0].numpy())

1


### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance)

In [6]:
AUTOTUNE = tf.data.AUTOTUNE


train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
# The model
EMBEDDING_DIM = 16

model = tf.keras.Sequential([
    layers.Embedding(MAX_FEATURE +1, EMBEDDING_DIM), # 10000 x 16 (batch, sequence, embedding)
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(), # I thing average over embedding of the sqeunce(250 tokens)
    layers.Dropout(0.2), 
    layers.Dense(4) # 4 classes
    
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 4)                 68        
                                                                 
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
__________________________________________________

In [8]:
model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # sparse means the classes not vecotrs
    # i.e [0, 0, 1, 0] insted they a number i.e: 2
)

In [9]:
# Training
EPOCHS = 10
history = model.fit(train_ds,
                   validation_data = val_ds,
                   epochs = EPOCHS)

Epoch 1/10
200/200 [==============================] - 2s 5ms/step - loss: 1.3777 - accuracy: 0.3480 - val_loss: 1.3674 - val_accuracy: 0.3837
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 1.3474 - accuracy: 0.4661 - val_loss: 1.3278 - val_accuracy: 0.5169
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 1.2961 - accuracy: 0.5322 - val_loss: 1.2693 - val_accuracy: 0.5813
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 1.2281 - accuracy: 0.5869 - val_loss: 1.1993 - val_accuracy: 0.6206
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 1.1530 - accuracy: 0.6378 - val_loss: 1.1260 - val_accuracy: 0.6531
Epoch 6/10
200/200 [==============================] - 1s 4ms/step - loss: 1.0776 - accuracy: 0.6831 - val_loss: 1.0552 - val_accuracy: 0.6925
Epoch 7/10
200/200 [==============================] - 1s 4ms/step - loss: 1.0082 - accuracy: 0.7145 - val_loss: 0.9910 - val_accuracy: 0.7231
Epoch 

In [10]:
# evaluating the model
loss, accuracy = model.evaluate(test_ds, verbose=2)

250/250 - 1s - loss: 0.8692 - accuracy: 0.7374 - 745ms/epoch - 3ms/step
